In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install tensorflow==2.18.0

In [4]:
import numpy as np
import pandas as pd
import joblib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [5]:
# Load dataset
df = pd.read_excel('/content/drive/MyDrive/lstm deeplearning training/crop yield data sheet.xlsx')
df.dropna(inplace=True)  # Remove missing values

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Rain Fall (mm),99.0,849.797980,400.042676,400.0,450.0,1150.0,1237.5,1300.0
Fertilizer,99.0,67.808081,10.028202,50.0,60.0,70.0,77.0,80.0
Nitrogen (N),99.0,70.737374,6.677079,59.0,65.0,71.0,77.0,80.0
Phosphorus (P),99.0,21.131313,1.951695,18.0,19.5,21.0,23.0,25.0
Potassium (K),99.0,18.060606,1.817254,15.0,16.0,19.0,19.0,22.0
Yeild (Q/acre),99.0,9.050505,1.965902,5.5,7.0,9.0,11.0,12.0


In [7]:

# Define features and target
X = df[['Fertilizer', 'Temperatue', 'Nitrogen (N)', 'Phosphorus (P)', 'Potassium (K)']]
y = df['Yeild (Q/acre)']

In [8]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [9]:
# Scaling data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
# Reshape input for LSTM (samples, time steps, features)
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

In [11]:
# Build LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(1, X_train_scaled.shape[2])),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)  # Output layer for regression
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

In [15]:

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=60, batch_size=8, validation_data=(X_test_scaled, y_test))

Epoch 1/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 2.1251 - mae: 1.1481 - val_loss: 1.9131 - val_mae: 0.9687
Epoch 2/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 2.5263 - mae: 1.3004 - val_loss: 1.8631 - val_mae: 0.9444
Epoch 3/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 2.0244 - mae: 1.1388 - val_loss: 1.8235 - val_mae: 0.9223
Epoch 4/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 2.5678 - mae: 1.1846 - val_loss: 1.8355 - val_mae: 0.9320
Epoch 5/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 2.1389 - mae: 1.1460 - val_loss: 1.8346 - val_mae: 0.9331
Epoch 6/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 1.8237 - mae: 1.1068 - val_loss: 1.7804 - val_mae: 0.9086
Epoch 7/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 2.4742 - mae: 1.2379 - val_loss: 1.7448 - val_mae: 0.8932
Epoch 8/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 1.8477 - mae: 1.0993 - val_loss: 1.7587 - val_mae: 0.9005
Epoch 9/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2.270

In [16]:
# Evaluate model
test_loss, test_mae = model.evaluate(X_test_scaled, y_test)
print("Test Loss (MSE):", test_loss)
print("Test MAE:", test_mae)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 1.1478 - mae: 0.7058
Test Loss (MSE): 1.1478137969970703
Test MAE: 0.705792248249054


In [17]:
%cd '/content/drive/MyDrive/lstm deeplearning training'

/content/drive/MyDrive/lstm deeplearning training


In [18]:
# Save the trained LSTM model & scaler
model.save('lstm_crop_yieldfinal.h5')
joblib.dump(scaler, 'scaler_lstmfinal.pkl')
joblib.dump(list(X.columns), 'feature_names_lstmfinal.pkl')

['feature_names_lstmfinal.pkl']

In [19]:
# ----------------- Prediction -----------------
# Load the saved model with custom objects
loaded_model = tf.keras.models.load_model(
    'lstm_crop_yieldfinal.h5',
    custom_objects={'mse': tf.keras.losses.MeanSquaredError()}
)

# Load the scaler and feature names
scaler = joblib.load('scaler_lstmfinal.pkl')
feature_names = joblib.load('feature_names_lstmfinal.pkl')


In [20]:

# New data for prediction
new_data = pd.DataFrame([[73, 29, 73, 23, 20]], columns=feature_names)
new_data_scaled = scaler.transform(new_data)
new_data_scaled = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))

# Predict yield
predicted_yield = loaded_model.predict(new_data_scaled)
print("Predicted Yield (Q/acre):", predicted_yield[0][0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step
Predicted Yield (Q/acre): 10.091887


In [21]:
pip freeze>req.txt


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import pickle

In [ ]:
# Load dataset
df = pd.read_csv("updated_dataset.csv")

In [ ]:
# Encode 'label' (Crop Name)
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])

In [ ]:
# Save label encoder
with open("label_encoder.pkl", "wb") as file:
    pickle.dump(label_encoder, file)

In [ ]:
# One-hot encode 'Location' (to keep text in X)
df = pd.get_dummies(df, columns=["Location"], dtype=int)

In [ ]:
# Normalize numerical features
scaler = MinMaxScaler()
X = scaler.fit_transform(df.drop(columns=["label"]))
y = np.array(df["label"])

In [ ]:
# Save scaler
with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

In [ ]:
# Reshape for LSTM
X = X.reshape((X.shape[0], 1, X.shape[1]))

In [ ]:
# Define LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation="relu"),
    Dense(len(label_encoder.classes_), activation="softmax")  # Output layer
])


In [ ]:
# Compile model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])


In [ ]:

# Train model
model.fit(X, y, epochs=65, batch_size=16, validation_split=0.2)

Epoch 1/65
110/110 [==============================] - 1s 6ms/step - loss: 0.1126 - accuracy: 0.9625 - val_loss: 24.9723 - val_accuracy: 0.0841
Epoch 2/65
110/110 [==============================] - 1s 6ms/step - loss: 0.1050 - accuracy: 0.9653 - val_loss: 24.9301 - val_accuracy: 0.0818
Epoch 3/65
110/110 [==============================] - 1s 5ms/step - loss: 0.1157 - accuracy: 0.9648 - val_loss: 25.0130 - val_accuracy: 0.0818
Epoch 4/65
110/110 [==============================] - 1s 7ms/step - loss: 0.0959 - accuracy: 0.9705 - val_loss: 25.5233 - val_accuracy: 0.0750
Epoch 5/65
110/110 [==============================] - 1s 5ms/step - loss: 0.1032 - accuracy: 0.9602 - val_loss: 25.6475 - val_accuracy: 0.0841
Epoch 6/65
110/110 [==============================] - 1s 5ms/step - loss: 0.0938 - accuracy: 0.9688 - val_loss: 25.5123 - val_accuracy: 0.0795
Epoch 7/65
110/110 [==============================] - 1s 5ms/step - loss: 0.0973 - accuracy: 0.9670 - val_loss: 25.2409 - val_accuracy: 0.0773

In [ ]:
# Save model
model.save("lstm_crop_model.h5")
print("Model saved as lstm_crop_model.h5")

Model saved as lstm_crop_model.h5


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle

# Load trained model
model = tf.keras.models.load_model("lstm_crop_model.h5")

# Load encoders
with open("label_encoder.pkl", "rb") as file:
    label_encoder = pickle.load(file)

with open("scaler.pkl", "rb") as file:
    scaler = pickle.load(file)

# Extract the original training feature names
original_features = scaler.feature_names_in_

# Function to preprocess new input data
def preprocess_input(data):
    # Convert input to DataFrame
    new_data = pd.DataFrame([data])

    # One-hot encode 'Location' (create all possible location columns)
    new_data = pd.get_dummies(new_data, columns=["Location"])

    # Add missing one-hot encoded location columns with value 0
    for col in original_features:
        if col not in new_data.columns:
            new_data[col] = 0  # Add missing column with default 0

    # Reorder columns to match training order
    new_data = new_data[original_features]

    # Normalize input
    new_data = scaler.transform(new_data)

    # Reshape for LSTM
    return new_data.reshape((1, 1, new_data.shape[1]))

# Function to predict crop
def predict_crop(input_data):
    processed_data = preprocess_input(input_data)
    pred = model.predict(processed_data)
    predicted_crop = label_encoder.inverse_transform([np.argmax(pred)])
    return predicted_crop[0]

# Example input (including Latitude & Longitude)
input_data = {
    "N": 27, "P": 76, "K": 83, "temperature": 19.12829388,
    "humidity": 14.92241479, "ph": 6.289614016, "rainfall": 89.61857826,
    "Latitude": 26.23011, "Longitude": 92.94331,  # Included Latitude & Longitude
    "Location": "Assam"  # Text format
}

# Get prediction
predicted_crop = predict_crop(input_data)
print("Predicted Crop:", predicted_crop)


1/1 [==============================] - 1s 1s/step
Predicted Crop: chickpea
